In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import mlflow
import mlflow.sklearn

def load_data():
    print("📥 Loading churn dataset...")
    df = pd.read_csv(r'D:\Github_Share\Dataset\WA_Fn-UseC_-Telco-Customer-Churn.csv')

    print("🧹 Cleaning and preprocessing data...")
    df = df[df["TotalCharges"].str.strip() != ""]
    df["TotalCharges"] = pd.to_numeric(df["TotalCharges"])
    df.drop(["customerID"], axis=1, inplace=True)
    df["Churn"] = df["Churn"].map({"Yes": 1, "No": 0})
    df = pd.get_dummies(df)

    selected_features = [
        "tenure",
        "MonthlyCharges",
        "TotalCharges",
        "Contract_Month-to-month",
        "InternetService_Fiber optic",
        "TechSupport_No",
        "OnlineSecurity_No",
        "PaymentMethod_Electronic check"
    ]

    for col in selected_features:
        if col not in df.columns:
            df[col] = 0

    X = df[selected_features]
    y = df["Churn"]

    print("✅ Features prepared. Proceeding to split...")
    return train_test_split(X, y, test_size=0.2, random_state=42)

def train_and_log():
    X_train, X_test, y_train, y_test = load_data()

    print("🚀 Training Logistic Regression model...")
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)

    print("📊 Evaluating model...")
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"🎯 Accuracy: {acc:.4f}")

    print("📝 Logging model and metrics to MLflow...")
    mlflow.set_experiment("Churn-Prediction-LogisticRegression")
    input_example = X_test.iloc[:1]

    with mlflow.start_run():
        mlflow.sklearn.log_model(model, "model", input_example=input_example)
        mlflow.log_metric("accuracy", acc)
        mlflow.log_params(model.get_params())
        print("✅ Model logged to MLflow successfully.")

if __name__ == "__main__":
    train_and_log()